In [3]:
# Install required libraries
!pip install transformers torch polars faiss-cpu gradio gdown

import os
import numpy as np
import torch
import faiss
import gdown
import zipfile
import polars as pl
import gradio as gr
from transformers import CLIPTokenizer, CLIPModel
from PIL import Image
from io import BytesIO

# Define Google Drive paths
EMBEDDING_GDRIVE_LINK = "https://drive.google.com/uc?id=1PsIyjyFvNMqlvPpeHUMtxMbc2wvWm9-h"
IMAGES_ZIP_GDRIVE_LINK = "https://drive.google.com/uc?id=1TeK8r_IMsdhMPm3K-0FK2-To5IWiCBch"

# Download embeddings file
EMBEDDING_CSV_PATH = "/content/image_embeddings.csv"
gdown.download(EMBEDDING_GDRIVE_LINK, EMBEDDING_CSV_PATH, quiet=False)

# Download and extract images
IMAGES_ZIP_PATH = "/content/images.zip"
IMAGES_FOLDER_PATH = "/content/images"

gdown.download(IMAGES_ZIP_GDRIVE_LINK, IMAGES_ZIP_PATH, quiet=False)

with zipfile.ZipFile(IMAGES_ZIP_PATH, "r") as zip_ref:
    zip_ref.extractall(IMAGES_FOLDER_PATH)

# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

# Load dataset
df = pl.read_csv(EMBEDDING_CSV_PATH)
df = df.with_row_index()




Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 4, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/main.py", line 11, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_indentation
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/util

KeyboardInterrupt: 

In [ ]:
def parse_embedding(embedding_str):
    return np.array([float(x) for x in embedding_str.strip("[]").split(", ")])

image_embeddings = np.vstack([parse_embedding(x) for x in df["Image Embedding"].to_list()]).astype(np.float32)

# FAISS index for similarity search
d = image_embeddings.shape[1]
index = faiss.IndexFlatIP(d)
index.add(image_embeddings)

# Function to get text embedding
def get_text_embedding(text):
    inputs = tokenizer(text, return_tensors="pt").to(device)
    with torch.no_grad():
        text_embedding = model.get_text_features(**inputs).cpu().numpy().flatten()
    return text_embedding

IMAGES_FOLDER_PATH = "/content/images/kaggle/working/dsd_images"
# Function to fetch top 10 image indices
def get_top_images(query):
    query_embedding = get_text_embedding(query).reshape(1, -1)
    _, I = index.search(query_embedding, 10)

    top_10_indices = df[I[0]]["index"]
    top_10_images = [os.path.join(IMAGES_FOLDER_PATH, f"{idx}.jpg") for idx in top_10_indices]

    return top_10_images

# Gradio UI
def search_images(query):
    image_paths = get_top_images(query)
    images = [Image.open(path) for path in image_paths]
    return images

iface = gr.Interface(
    fn=search_images,
    inputs="text",
    outputs=gr.Gallery(label="Top 10 Matching Images"),
    title="Visual Search with CLIP",
    description="Enter a text query and retrieve the top 10 matching images from the dataset."
)

iface.launch(share=True, debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://bd4461a069e9d2aa92.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
